In [1]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST=neo4j+s://d9ea9625.databases.neo4j.io
%env USER=neo4j
%env PW=3SCMw1FEgMeBc-B0zhAVQ-9LKh4itXkx1sWeGeNqHrA

env: CYPHER=/usr/bin/cypher-shell
env: HOST=neo4j+s://d9ea9625.databases.neo4j.io
env: USER=neo4j
env: PW=3SCMw1FEgMeBc-B0zhAVQ-9LKh4itXkx1sWeGeNqHrA


In [2]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [3]:
!{CONNECT} "show users"

+------------------------------------------------------------------+
| user    | roles      | passwordChangeRequired | suspended | home |
+------------------------------------------------------------------+
| "neo4j" | ["PUBLIC"] | FALSE                  | FALSE     | NULL |
+------------------------------------------------------------------+

1 row
ready to start consuming query after 9 ms, results consumed after another 59 ms


In [4]:
from IPython.core.magic import register_cell_magic
import shlex

@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

# 1. Operation: Return the unique node labels and the number of nodes for each

In [5]:
!{CONNECT} "CALL db.schema.visualization()"

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| nodes                                                                                                                                                                                                                                                                                                                  | relationships                                                                                                                                          |
+---------------------------------------------------------------

In [6]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(labels(n))"

+-----------------------------------+
| labels(n)      | count(labels(n)) |
+-----------------------------------+
| ["Recipe"]     | 11634            |
| ["Author"]     | 303              |
| ["Ingredient"] | 3077             |
| ["DietType"]   | 12               |
| ["Collection"] | 1049             |
+-----------------------------------+

5 rows
ready to start consuming query after 36 ms, results consumed after another 24 ms


# 2. Operation: Return properties of the nodes



later Return the unique relationship types and the number of relationships for each

In [7]:
!{CONNECT} "MATCH (n) UNWIND keys(n) as key RETURN DISTINCT labels(n), collect(distinct(key))"

#STYLE and visualizations??? not included in propererties but in the graph 

+------------------------------------------------------------------------------------------------+
| labels(n)      | collect(distinct(key))                                                        |
+------------------------------------------------------------------------------------------------+
| ["Recipe"]     | ["id", "name", "cookingTime", "preparationTime", "description", "skillLevel"] |
| ["Author"]     | ["name"]                                                                      |
| ["Ingredient"] | ["name"]                                                                      |
| ["DietType"]   | ["name"]                                                                      |
| ["Collection"] | ["name"]                                                                      |
+------------------------------------------------------------------------------------------------+

5 rows
ready to start consuming query after 38 ms, results consumed after another 77 ms


# 3. Return the unique relationship types and the number of relationships for each 

In [8]:
!{CONNECT} "MATCH ()-[r]->() RETURN DISTINCT type(r), count(r)"

+----------------------------------+
| type(r)               | count(r) |
+----------------------------------+
| "WROTE"               | 11634    |
| "CONTAINS_INGREDIENT" | 106148   |
| "DIET_TYPE"           | 14858    |
| "COLLECTION"          | 32240    |
+----------------------------------+

4 rows
ready to start consuming query after 35 ms, results consumed after another 226 ms


# 4: Logical or, return properties of relationships. Retrieves recipes authored by "Sarah Cook" or "Silvana Franco." Returns author names, recipe names, and relationship properties. Results are ordered by recipe names in ascending order and limited to the top 10.

In [9]:
!{CONNECT} "MATCH (a:Author)-[w:WROTE]->(r:Recipe) WHERE (a.name = 'Sarah Cook' OR a.name = 'Silvana Franco') RETURN a.name as author, r.name as recipe_name, id(w) ORDER BY r.name ASC LIMIT 10"

+--------------------------------------------------------------------------------------+
| author           | recipe_name                                               | id(w) |
+--------------------------------------------------------------------------------------+
| "Silvana Franco" | "After-dinner mint cream"                                 | 3280  |
| "Sarah Cook"     | "Afternoon tea churros"                                   | 9759  |
| "Sarah Cook"     | "All-in-one posh lamb balti"                              | 9936  |
| "Sarah Cook"     | "All-in-one spring roast chicken"                         | 9918  |
| "Sarah Cook"     | "Almond & honey pastries with orange cream"               | 2887  |
| "Sarah Cook"     | "Almond florentines"                                      | 8584  |
| "Sarah Cook"     | "Amatriciana chicken traybake"                            | 3575  |
| "Sarah Cook"     | "Aperol & limoncello cocktail"                            | 1396  |
| "Sarah Cook"     | 

# 5. Logical and, negation, in descending order. Find the query with cooking time <= 600 and prepation time <=600, AND NOT skill level = A CHALLENGE"

In [10]:
# LOGICAL AND STMT, descending order 
!{CONNECT} "MATCH (r), (r) WHERE r.cookingTime <= 600 AND r.preparationTime <= 600  RETURN r.name as recipe_name, r.preparationTime as prep_time, r.cookingTime as cook_time, r.skillLevel as skill ORDER BY r.preparationTime DESC LIMIT 10 "

+--------------------------------------------------------------------------------+
| recipe_name                                   | prep_time | cook_time | skill  |
+--------------------------------------------------------------------------------+
| "Spicy tuna quinoa salad"                     | 600       | 600       | "Easy" |
| "Christmas mess"                              | 600       | 300       | "Easy" |
| "Herby slashed roasties"                      | 600       | 0         | "Easy" |
| "Gremolata prawn pasta"                       | 600       | 600       | "Easy" |
| "Citrus peach cooler"                         | 600       | 0         | "Easy" |
| "Chocolate tea-pots"                          | 600       | 300       | "Easy" |
| "Chipotle chicken tacos with pineapple salsa" | 600       | 600       | "Easy" |
| "Oriental beef skewers with cucumber salad"   | 600       | 360       | "Easy" |
| "Tom Collins"                                 | 600       | 120       | "Easy" |
| "T

# 6. Logical OR, containing negation, desc order.Retrieves recipes authored by either "James Martin" or "Diana Henry" with a skill level not set to "Hard." Returns author names and recipe names, ordered by author names in descending order.

In [11]:
!{CONNECT} "MATCH (a:Author)-[w:WROTE]->(r:Recipe) WHERE (a.name = 'James Martin' OR a.name = 'Diana Henry') AND NOT r.skillLevel = 'Hard' RETURN a.name as author , r.name as recipe_name ORDER BY a.name DESC LIMIT 10"

+-----------------------------------------------------------------------+
| author         | recipe_name                                          |
+-----------------------------------------------------------------------+
| "James Martin" | "Star anise biscuits"                                |
| "James Martin" | "Grilled dab with garlic butter"                     |
| "James Martin" | "Roasted butternut squash soup"                      |
| "James Martin" | "Herby lemon chicken with Tuscan beans"              |
| "James Martin" | "Halloumi & pepper kebabs with lemony cabbage salad" |
| "James Martin" | "Almond & apricot trifles"                           |
| "James Martin" | "Rib-eye steaks with chilli butter & homemade chips" |
| "James Martin" | "Chicken stuffed with goatâ€™s cheese & tarragon"    |
| "James Martin" | "Frosted white chocolate Easter cake"                |
| "James Martin" | "Baked pumpkin with cream & onions"                  |
+-------------------------------------

# 7. 3 relational types. Find gluten-free recipes by the author Good Food and has olive oil. 

In [12]:
!{CONNECT} "MATCH (p:Recipe)-[r:DIET_TYPE]->(m:DietType), (a: Author)-[aa]->(aaa: Recipe), (rr: Recipe)-[rrr]->(rrrr: Ingredient) WHERE m.name = 'Gluten-free' AND a.name = 'Good Food' AND rrrr.name = 'olive oil' return p.name as recipe, r, m, rrrr LIMIT 5"


+--------------------------------------------------------------------------------------------------------------------------------+
| recipe                                  | r            | m                                 | rrrr                              |
+--------------------------------------------------------------------------------------------------------------------------------+
| "Baked glazed ham"                      | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "olive oil"}) |
| "Honeydew melon, cucumber & lime juice" | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "olive oil"}) |
| "Pumpkin purÃ©e "                       | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "olive oil"}) |
| "Gooseberry & mint lemonade"            | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "olive oil"}) |
| "Spiced pear chutney"                   | [:DIET_TYPE] | (:DietType {name: "Glute

# 8. 3 relational types. Finds recipes containing specific ingredients, belonging to a collection, and authored by a particular author. Returns recipe names, ingredient names, collection names, and author names. Find gluten-free recipes by the author Good Food and has butter. 

In [13]:
!{CONNECT} "MATCH (p:Recipe)-[r:DIET_TYPE]->(m:DietType), (a: Author)-[aa]->(aaa: Recipe), (rr: Recipe)-[rrr]->(rrrr: Ingredient) WHERE m.name = 'Gluten-free' AND a.name = 'Good Food' AND rrrr.name = 'butter' return p.name as recipe, r, m, rrrr LIMIT 5"


+-----------------------------------------------------------------------------------------------------------------------------+
| recipe                                  | r            | m                                 | rrrr                           |
+-----------------------------------------------------------------------------------------------------------------------------+
| "Baked glazed ham"                      | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "butter"}) |
| "Honeydew melon, cucumber & lime juice" | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "butter"}) |
| "Pumpkin purÃ©e "                       | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "butter"}) |
| "Gooseberry & mint lemonade"            | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredient {name: "butter"}) |
| "Spiced pear chutney"                   | [:DIET_TYPE] | (:DietType {name: "Gluten-free"}) | (:Ingredi

In [14]:
# worked in Neo4js
#!{CONNECT} "MATCH (a:Recipe)-[:CONTAINS_INGREDIENT]->(b:Ingredient {name: 'tomato'}), (a)-[:COLLECTION]->(c:Collection {name: '200-400 calorie'}), (a)<-[:WROTE]-(d:Author {name: 'Sara Buenfeld'}) RETURN a.name as recipe_names, b.name as contain_ingredient, c.name as collection, d.name as author"

# 9. Logical or, containing negation, asc order. Find recipes that are either gluten-free or have a prep time <= 600, where author is not Tony Tobin. Recipes are ordered by prep time. 

In [15]:
#OFFICial: Logical or, containing negation, asc order
!{CONNECT} "MATCH (p:Recipe)-[r:DIET_TYPE]->(m:DietType) WHERE (m.name = 'Gluten-free' OR p.preparationTime <= 600) WITH collect(m.name) as recipe MATCH (a: Author)-[aa]->(aaa: Recipe) WHERE NOT a.name = 'Tony Tobin' RETURN DISTINCT a.name as author, aa as relation, aaa ORDER BY aaa.preprationTime DESC LIMIT 5"


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| author        | relation | aaa                                                                                                                                                                                                                                                                             |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Jane Hornby" | [:WROTE] | (:Recipe {name: "Honey-mustard chicken pasta", preparationTime

# 10. Find recipes where 1000 > cooking time is > 1 and prep time is <= 600, skill level is easy and orderby ascending order for prep time.

In [16]:
#OFFICIAL for logical and, not, ascending order 
!{CONNECT} "MATCH (r), (r), (r), (r) WHERE (1 < r.cookingTime < 1000) AND r.preparationTime <= 600 AND (NOT r.skillLevel = 'Easy' AND r.skillLevel = 'More effort') RETURN r.name as recipe, r.preparationTime as prep_time, r.cookingTime as cook_Time ORDER BY r.preparationTime ASC LIMIT 10 "

+---------------------------------------------------------------------------------+
| recipe                                                  | prep_time | cook_Time |
+---------------------------------------------------------------------------------+
| "Passion fruit curd"                                    | 0         | 900       |
| "Glam trifle slice"                                     | 0         | 600       |
| "Caramel sauce"                                         | 120       | 600       |
| "Spiced pecans & peach fizz"                            | 120       | 300       |
| "Salted almond brittle"                                 | 120       | 600       |
| "Strawberry & sweet wine savarin"                       | 120       | 900       |
| "Purple sprouting broccoli, poached eggs & hollandaise" | 300       | 600       |
| "Rich nutmeg ice cream"                                 | 300       | 600       |
| "Milk chocolate sorbet"                                 | 300       | 900 

In [17]:
# 10. Logical and, containing negation, desc order. Filters recipes based on conditions, excluding allergenic ingredients while ensuring the presence of "prawn" and "potato." Returns recipe names and their ingredient names, ordered by recipe names in descending order.

#!{CONNECT} "MATCH (r:Recipe) WHERE ALL(allergies IN ['milk', 'butter', 'cream', 'soft cheese'] WHERE NOT (r)-[:CONTAINS_INGREDIENT]->(:Ingredient {name: allergies})) AND ALL(ingredientName IN ['prawn', 'potato'] WHERE (r)-[:CONTAINS_INGREDIENT]->(:Ingredient {name:ingredientName})) RETURN r.name AS recipe, [(r)-[:CONTAINS_INGREDIENT]->(i) | i.name] AS ingredients ORDER BY r.name DESC"


# EXTRA CREDIT (sorry some are numbered wrong) 

# 1. Create a new node for each existing node label with at least one property

In [20]:
%%neo 
CREATE (:Author {name: 'Mom'})

0 rows
ready to start consuming query after 11 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels


In [35]:
%%neo 
CREATE (:Recipe {name: 'Home-cooked foods'})

0 rows
ready to start consuming query after 31 ms, results consumed after another 0 ms
Added 1 nodes, Set 1 properties, Added 1 labels


# 2. Create a relationship for each existing relationship type with at least one property

In [31]:
%%neo 
MATCH (a: Author {name: 'Mom'})
MATCH (r: Recipe {name: 'Home-cooked foods'})
CREATE (a)-[:WROTE]->(r); 


0 rows
ready to start consuming query after 70 ms, results consumed after another 0 ms


In [43]:
%%neo 
MATCH (a: Author)-[g]->(r), (a: Author)-[g]->(r)
WHERE a.name = 'Mom' AND r.name = 'Home-cooked foods'
RETURN a, r;

Driver execution failed


# 3. Update the properties of one or more nodes

In [ ]:
%%neo 
MATCH (a: Author {name: "Mom"})
SET a.chef_status = True, 
    a.cool_chef_on_block = True
RETURN a.name, a.chef_status, a.cool_chef_on_block; 

# 5. Update the properties of one or more relationships

# 6. Add a new node label to an existing node

In [42]:
%%neo
MATCH (n {name: 'Grandpa'})
SET n: "Best Grandpa Cook"
RETURN n.name, labels(n) as labels; 

Driver execution failed


# 7. Update the relationship type of at least one relationship

In [ ]:
%%neo



In [41]:
%%neo 
MATCH (a: Author)-[w: WROTE]->(p: recipe) 
MERGE (a)-[wr: WROTE_REMIX]-> (r)
DELETE w;

Driver execution failed


# 9. Delete at least one relationship based on some criteria

In [38]:
%%neo 
MATCH (p:Person)-[r: COLLECTIONS]->() 
DELETE r; 

Driver execution failed


# 10. Delete at least one node based on some criteria

In [39]:
%%neo 
MATCH (c: Collections)
DELETE c;

Driver execution failed
